<a href="https://colab.research.google.com/github/Dt-Crxs/UAIC/blob/main/UAIC_BlueWhale5Hz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Dec 25 13:46:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from tqdm import tqdm
import json
import glob
import re
import pandas as pd
import random
import cv2
import shutil

#Setup

In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR

fatal: destination path 'PaddleOCR' already exists and is not an empty directory.


In [ ]:
%cd /content/drive/MyDrive/PaddleOCR

/content/drive/MyDrive/PaddleOCR


In [ ]:
!python -m pip install paddlepaddle-gpu==0.0.0.post112 -f https://www.paddlepaddle.org.cn/whl/linux/gpu/develop.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://www.paddlepaddle.org.cn/whl/linux/gpu/develop.html
     |████████████████████████████████| 586.7 MB 28 kB/s 
     |████████████████████████████████| 385 kB 39.6 MB/s 


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 619 kB 33.8 MB/s 
     |████████████████████████████████| 4.9 MB 57.7 MB/s 
     |████████████████████████████████| 2.2 MB 62.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 8.8 MB 62.4 MB/s 
     |████████████████████████████████| 132 kB 76.5 MB/s 
     |████████████████████████████████| 204 kB 71.3 MB/s 
     |████████████████████████████████| 399 kB 68.1 MB/s 
     |████████████████████████████████| 2.3 MB 59.1 MB/s 
  Created wheel for lanms-neo: filename=lanms_neo-1.0.2-cp38-cp38-linux_x86_64.whl size=110775 sha256=62d3fc778fc1233ddad14c7093002e65740c20432c573b950e7044b95e72df17
  Stored in directory: /root/.cache/pip/wheels/e2/a4/df/d69223851b1eb3a4d1b961ae86a9c08bba24412cd3939c5850
  Created wheel for Polygon3: filenam

#Download data

In [ ]:
!pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
## Download training data
!gdown --id 1NJJA1A8I2Xj5-107E3DFohBNzjWyaaf7
!unzip uaic2022_training_data_update.zip -d dataset

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Access denied with the following error:

 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1NJJA1A8I2Xj5-107E3DFohBNzjWyaaf7 

unzip:  cannot find or open uaic2022_training_data_update.zip, uaic2022_training_data_update.zip.zip or uaic2022_training_data_update.zip.ZIP.


In [ ]:
## Download Public Valid data
!gdown --id 1frwoaHR24ETHKf4aBQbce6wSoZO-Bi80
!unzip uaic2022_public_valid.zip -d dataset

#Train

## Train model detection


### Preprocess for train model detection

In [ ]:
def convert(src):
    f = open(src)
    label = json.load(f)
    for item in label:
      item['transcription'] = item['text']
      del item['text']
    label_str = json.dumps(label)
    return label_str

Convert to format PaddleOCR

In [ ]:
root_img = './dataset/uaic2022_training_data/images/'
root_label = glob.glob('./dataset/uaic2022_training_data/labels/*')
random.shuffle(root_label)
num_imgs = len(root_label)

train_ratio = 0.7
val_ratio = 0.3

train_idx_start = int(num_imgs*train_ratio)
val_idx_start = int(num_imgs*(train_ratio))

os.makedirs('./dataset/det/train')
os.makedirs('./dataset/det/val')

train_label = open("./dataset/det/train_label.txt","w")
val_label = open("./dataset/det/val_label.txt","w")

output = './dataset/det'

for idx, file in enumerate(root_label):
    data = convert(file)
    img_name = os.path.basename(file).split('.')[0] + '.jpg'

    if idx < val_idx_start:
      shutil.copy(os.path.join(root_img,img_name), os.path.join(output,'train',img_name))
      train_label.write( 'train/'+img_name+ '\t'+f'{data}' + '\n')
    else:
      shutil.copy(os.path.join(root_img,img_name), os.path.join(output,'val',img_name))
      val_label.write( 'val/'+img_name+ '\t'+f'{data}' + '\n')

FileExistsError: ignored

Download pretrain 

In [ ]:
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.1/en_det/det_r50_db%2B%2B_icdar15_train.tar -P ./pretrain_models
!tar -xf ./pretrain_models/det_r50_db++_icdar15_train.tar -C ./pretrain_models

--2022-12-24 07:01:31--  https://paddleocr.bj.bcebos.com/dygraph_v2.1/en_det/det_r50_db%2B%2B_icdar15_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211712000 (202M) [application/x-tar]
Saving to: ‘./pretrain_models/det_r50_db++_icdar15_train.tar.1’

det_r50_db++_icdar1 100%[===================>] 201.90M  11.9MB/s    in 37s     

utime(./pretrain_models/det_r50_db++_icdar15_train.tar.1): No such file or directory
2022-12-24 07:02:10 (5.41 MB/s) - ‘./pretrain_models/det_r50_db++_icdar15_train.tar.1’ saved [211712000/211712000]



### Training

In [ ]:
!python3 tools/train.py -c ./configs/det/det_r50_db++_icdar15.yml -o   \
         Global.pretrained_model=./pretrain_models/det_r50_db++_icdar15_train/best_accuracy  \
         Global.epoch_num=3  \
         Global.save_model_dir=/content/drive/MyDrive/uaic2022  \
         Global.save_epoch_step=2 \
        Train.dataset.data_dir=./dataset/det/  \
        Train.dataset.label_file_list=['./dataset/det/train_label.txt'] \
        Train.loader.batch_size_per_card=8  \
        Eval.dataset.data_dir=./dataset/det/  \
        Eval.dataset.label_file_list=['./dataset/det/val_label.txt'] 

        

##Train model recognition

### Preprocess for train model recognition

In [ ]:
def order_points(pts):
    if isinstance(pts, list):
        pts = np.asarray(pts, dtype='float32')
    rect = np.zeros((4, 2), dtype='float32')
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def perspective_transform(img, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]], dtype = "float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(img, M, (maxWidth, maxHeight))
    return warped   

In [ ]:
def crop(train_label_path, output_path):
  train_label = open(train_label_path,'r') 
  train_file_list = train_label.readlines()
  crop_label = open(output_path, 'w', encoding="utf-8")

  count=0
  for line in tqdm(train_file_list):
      file_name, labels = line.strip().split('\t')
      labels = json.loads(labels)
      img_path = os.path.join(root_dir, file_name)
      image = cv2.imread(img_path)
      id=0
      for label in labels:
        pts = label['points']
        transcription = label['transcription']

        try:
          pts = [[int(x[0]), int(x[1])] for x in pts]
        except:
          continue

        if transcription == "###" or transcription == '#':
          continue
        
        cropped_img = perspective_transform(image, pts)
        img_crop_name = os.path.basename(file_name).split('.')[0] + '_' + str(id) + '.jpg'
        
        cv2.imwrite(os.path.join(os.path.dirname(output_path)  ,'img', img_crop_name), cropped_img)
        crop_label.write(f"img/{img_crop_name}\t{transcription}\n")
        id += 1
        count += 1
      
  print('Total box:', count)



Crop text box

In [ ]:
root_dir = '/content/drive/MyDrive/PaddleOCR/dataset/det'

outpath = '/content/drive/MyDrive/PaddleOCR/dataset/rec_data'

if (not os.path.exists(outpath + '/img')):
  os.makedirs(outpath + '/img')

output_train=os.path.join(outpath,'train_crop.txt')
crop('/content/drive/MyDrive/PaddleOCR/dataset/det/train_label.txt', output_train)

output_val=os.path.join(outpath,'val_crop.txt')
crop('/content/drive/MyDrive/PaddleOCR/dataset/det/val_label.txt', output_val)

In [ ]:
## Download pretrain 
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/rec_r50_vd_srn_train.tar -P ./pretrain_models
!tar -xf ./pretrain_models/rec_r50_vd_srn_train.tar -C ./pretrain_models

In [ ]:
## get dict vietnamese 
!wget https://raw.githubusercontent.com/hungcao0402/PaddleOCR-Vietnamese/master1/ppocr/utils/dict/vi_vietnam.txt

### Training

In [ ]:
!python3 tools/train.py -c ./configs/rec/rec_r50_fpn_srn.yml  \
                        -o Global.pretrained_model=./pretrain_models/rec_r50_vd_srn_train/best_accuracy  \
                            Global.character_dict_path=vi_vietnam.txt \
                            Global.epoch_num=4  \
                            Global.save_epoch_step=2 \
                            Train.dataset.name='SimpleDataSet'  \
                            Train.dataset.data_dir=/content/drive/MyDrive/PaddleOCR/dataset/rec_data  \
                            Train.dataset.label_file_list=['./dataset/rec_data/train_crop.txt'] \
                            Eval.dataset.name='SimpleDataSet'  \
                            Eval.dataset.data_dir=/content/drive/MyDrive/PaddleOCR/dataset/rec_data  \
                            Eval.dataset.label_file_list=['./dataset/rec_data/val_crop.txt'] 

Traceback (most recent call last):
  File "tools/train.py", line 205, in <module>
    config, device, logger, vdl_writer = program.preprocess(is_train=True)
  File "/content/drive/MyDrive/PaddleOCR/tools/program.py", line 625, in preprocess
    with open(os.path.join(save_model_dir, 'config.yml'), 'w') as f:
  File "/usr/lib/python3.8/_bootlocale.py", line 33, in getpreferredencoding
    def getpreferredencoding(do_setlocale=True):
KeyboardInterrupt
^C


#Evaluation

## Evaluate detection


In [ ]:
!python3 tools/eval.py -c ./configs/det/det_r50_db++_icdar15.yml  \
                       -o Global.pretrained_model=./pretrain_models/det_r50_db++_icdar15_train/best_accuracy  \
                          Eval.dataset.data_dir=./dataset/det  \
                          Eval.dataset.label_file_list=['./dataset/det/val_label.txt'] 

[2022/12/25 13:55:25] ppocr INFO: Architecture : 
[2022/12/25 13:55:25] ppocr INFO:     Backbone : 
[2022/12/25 13:55:25] ppocr INFO:         dcn_stage : [False, True, True, True]
[2022/12/25 13:55:25] ppocr INFO:         layers : 50
[2022/12/25 13:55:25] ppocr INFO:         name : ResNet
[2022/12/25 13:55:25] ppocr INFO:     Head : 
[2022/12/25 13:55:25] ppocr INFO:         k : 50
[2022/12/25 13:55:25] ppocr INFO:         name : DBHead
[2022/12/25 13:55:25] ppocr INFO:     Neck : 
[2022/12/25 13:55:25] ppocr INFO:         name : DBFPN
[2022/12/25 13:55:25] ppocr INFO:         out_channels : 256
[2022/12/25 13:55:25] ppocr INFO:         use_asf : True
[2022/12/25 13:55:25] ppocr INFO:     Transform : None
[2022/12/25 13:55:25] ppocr INFO:     algorithm : DB++
[2022/12/25 13:55:25] ppocr INFO:     model_type : det
[2022/12/25 13:55:25] ppocr INFO: Eval : 
[2022/12/25 13:55:25] ppocr INFO:     dataset : 
[2022/12/25 13:55:25] ppocr INFO:         data_dir : ./dataset/det
[2022/12/25 13:55

## Evaluate recognition


In [ ]:
!python3 tools/eval.py -c ./configs/rec/rec_r50_fpn_srn.yml  \
                       -o Global.checkpoints=./output/rec/srn_new/latest \
                          Global.character_type=ch  \
                          Global.character_dict_path=./vi_vietnam.txt \
                          Eval.dataset.name='SimpleDataSet'  \
                          Eval.dataset.data_dir=./dataset/rec_data  \
                          Eval.dataset.label_file_list=['./dataset/rec_data/val_crop.txt'] 

[2022/12/25 14:03:44] ppocr INFO: Architecture : 
[2022/12/25 14:03:44] ppocr INFO:     Backbone : 
[2022/12/25 14:03:44] ppocr INFO:         name : ResNetFPN
[2022/12/25 14:03:44] ppocr INFO:     Head : 
[2022/12/25 14:03:44] ppocr INFO:         hidden_dims : 512
[2022/12/25 14:03:44] ppocr INFO:         max_text_length : 25
[2022/12/25 14:03:44] ppocr INFO:         name : SRNHead
[2022/12/25 14:03:44] ppocr INFO:         num_decoder_TUs : 4
[2022/12/25 14:03:44] ppocr INFO:         num_encoder_TUs : 2
[2022/12/25 14:03:44] ppocr INFO:         num_heads : 8
[2022/12/25 14:03:44] ppocr INFO:     Transform : None
[2022/12/25 14:03:44] ppocr INFO:     algorithm : SRN
[2022/12/25 14:03:44] ppocr INFO:     in_channels : 1
[2022/12/25 14:03:44] ppocr INFO:     model_type : rec
[2022/12/25 14:03:44] ppocr INFO: Eval : 
[2022/12/25 14:03:44] ppocr INFO:     dataset : 
[2022/12/25 14:03:44] ppocr INFO:         data_dir : ./dataset/rec_data
[2022/12/25 14:03:44] ppocr INFO:         label_file_l

#Convert to inference model

In [ ]:
## Export detection model
!python3 tools/export_model.py -c ./configs/det/det_r50_db++_icdar15.yml  \
                               -o Global.pretrained_model=./pretrain_models/det_r50_db++_icdar15_train/best_accuracy \
                                  Global.save_inference_dir=./inference/det

W1225 15:50:07.140667 33229 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 11.2, Runtime API Version: 11.2
W1225 15:50:07.144438 33229 gpu_resources.cc:91] device: 0, cuDNN Version: 8.1.
[2022/12/25 15:50:09] ppocr INFO: load pretrain successful from ./pretrain_models/det_r50_db++_icdar15_train/best_accuracy
I1225 15:50:11.320317 33229 interpretercore.cc:239] New Executor is Running.
[2022/12/25 15:50:12] ppocr INFO: inference model is saved to ./inference/det/inference


In [ ]:
## Export recognition model
!python3 tools/export_model.py -c ./configs/rec/rec_r50_fpn_srn.yml  \
                               -o Global.pretrained_model=./output/rec/srn_new/latest \
                                  Global.character_dict_path=./vi_vietnam.txt \
                                  Global.save_inference_dir=./inference/rec

W1225 15:50:21.293617 33307 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 11.2, Runtime API Version: 11.2
W1225 15:50:21.296604 33307 gpu_resources.cc:91] device: 0, cuDNN Version: 8.1.
[2022/12/25 15:50:23] ppocr INFO: load pretrain successful from ./output/rec/srn_new/latest
I1225 15:50:28.870901 33307 interpretercore.cc:239] New Executor is Running.
[2022/12/25 15:50:31] ppocr INFO: inference model is saved to ./inference/rec/inference


Sau khi convert xong thì thử predict 

In [ ]:
#predict detection
!python3 tools/infer/predict_det.py --use_gpu=True \
                                    --det_model_dir="./inference/det"  \
                                    --image_dir=/content/drive/MyDrive/PaddleOCR/dataset/uaic2022_training_data/images/im0001.jpg

[2022/12/25 14:35:10] ppocr INFO: im0001.jpg	[[[193.0, 182.0], [214.0, 182.0], [214.0, 189.0], [193.0, 189.0]], [[222.0, 161.0], [296.0, 161.0], [296.0, 172.0], [222.0, 172.0]], [[182.0, 158.0], [219.0, 161.0], [219.0, 172.0], [182.0, 170.0]], [[164.0, 162.0], [181.0, 162.0], [181.0, 170.0], [164.0, 170.0]], [[138.0, 162.0], [161.0, 162.0], [161.0, 170.0], [138.0, 170.0]], [[272.0, 145.0], [286.0, 145.0], [286.0, 152.0], [272.0, 152.0]], [[215.0, 144.0], [237.0, 144.0], [237.0, 151.0], [215.0, 151.0]], [[146.0, 141.0], [179.0, 141.0], [179.0, 152.0], [146.0, 152.0]]]

[2022/12/25 14:35:10] ppocr INFO: 0 The predict time of /content/drive/MyDrive/PaddleOCR/dataset/uaic2022_training_data/images/im0001.jpg: 1.8949711322784424
[2022/12/25 14:35:10] ppocr INFO: The visualized image saved in ./inference_results/det_res_im0001.jpg


In [ ]:
#predict recognition
!python3 tools/infer/predict_rec.py \
          --image_dir="/content/drive/MyDrive/PaddleOCR/dataset/rec_data/img/im0000_1.jpg" \
          --rec_model_dir="./inference/rec/" --rec_image_shape="1,64,256" \
          --rec_algorithm="SRN"  \
          --rec_char_dict_path="./vi_vietnam.txt" --use_space_char=False

[2022/12/25 14:35:29] ppocr INFO: In PP-OCRv3, rec_image_shape parameter defaults to '3, 48, 320', if you are using recognition model with PP-OCRv2 or an older version, please set --rec_image_shape='3,32,320
[2022/12/25 14:35:31] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/dataset/rec_data/img/im0000_1.jpg:('KIA!', 0.8377001881599426)


#Text Detection and Recognition Inference Concatenation

In [ ]:
#get font to visualize
!gdown --id 1y5ijIAEOtHR20TCSKETO-9XHH-veCGID

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1y5ijIAEOtHR20TCSKETO-9XHH-veCGID
To: /content/drive/MyDrive/PaddleOCR/font-times-new-roman.ttf
100% 611k/611k [00:00<00:00, 64.2MB/s]


In [ ]:
!python3 ./tools/infer/predict_system.py  \
                    --use_gpu=True  \
                    --det_model_dir="./inference/det"  \
                    --rec_algorithm="SRN" \
                    --rec_model_dir="./inference/rec"  \
                    --rec_image_shape="1,64,256"  \
                    --drop_score=0.001  \
                    --image_dir="/content/drive/MyDrive/PaddleOCR/dataset/uaic2022_public_valid/images" \
                    --rec_char_dict_path="./vi_vietnam.txt" \
                    --use_space_char=False  \
                    --vis_font_path=font-times-new-roman.ttf 

[2022/12/25 15:51:22] ppocr INFO: In PP-OCRv3, rec_image_shape parameter defaults to '3, 48, 320', if you are using recognition model with PP-OCRv2 or an older version, please set --rec_image_shape='3,32,320
[2022/12/25 15:51:24] ppocr DEBUG: dt_boxes num : 2, elapse : 1.8711879253387451
[2022/12/25 15:51:24] ppocr DEBUG: rec_res num  : 2, elapse : 0.05168414115905762
[2022/12/25 15:51:24] ppocr DEBUG: 0  Predict time of /content/drive/MyDrive/PaddleOCR/dataset/uaic2022_public_valid/images/im0001.jpg: 1.925s
[2022/12/25 15:51:24] ppocr DEBUG: Natei, 0.721
[2022/12/25 15:51:24] ppocr DEBUG: CALAX, 0.919
[2022/12/25 15:51:24] ppocr DEBUG: The visualized image saved in ./inference_results/im0001.jpg
[2022/12/25 15:51:24] ppocr DEBUG: dt_boxes num : 0, elapse : 0.033399343490600586
[2022/12/25 15:51:24] ppocr DEBUG: rec_res num  : 0, elapse : 1.9073486328125e-06
[2022/12/25 15:51:24] ppocr DEBUG: 1  Predict time of /content/drive/MyDrive/PaddleOCR/dataset/uaic2022_public_valid/images/im000

# Evaluate E2E

In [ ]:
#convert to format paddle eval e2e
!python3 tools/end2end/convert_ppocr_label.py --mode=gt --label_path=./dataset/det/val_label.txt --save_folder=save_gt_label
!python3 tools/end2end/convert_ppocr_label.py --mode=pred --label_path=./content/drive/MyDrive/PaddleOCR/save_infer/system_results.txt --save_folder=save_infer

In [ ]:
!python3 tools/end2end/eval_end2end.py ./save_gt_label/ ./save_infer/

start testing...
Traceback (most recent call last):
  File "tools/end2end/eval_end2end.py", line 193, in <module>
    e2e_eval(gt_folder, pred_folder)
  File "tools/end2end/eval_end2end.py", line 91, in e2e_eval
    assert (len(parts) < 11)
AssertionError


# Convert To submit

Run inference cho tập test sau đó convert về format yêu cầu rồi zip lại và nộp

In [ ]:
!python3 tools/end2end/convert_ppocr_label.py --mode=pred \
                                              --label_path=./inference_results/system_results.txt \
                                              --save_folder=save_infer

The convert label saved in save_infer


In [ ]:
root = glob.glob('./save_infer/*')
os.mkdir('submit')
for file in root:
    file_name = os.path.basename(file).replace('.jpg','')
    f = open(file,'r')
    lines = f.readlines()

    output = open('submit/'+file_name,'w')
    for line in lines:
        line=line.strip()
        label = line.split('\t')
        bbox = label[:8]
        trans = label[-1]
        data = ','.join(bbox)+ ',' +trans+'\n'
        output.write(data)

In [ ]:
!zip -j submit.zip submit/*

  adding: im0001.txt (deflated 17%)
  adding: im0002.txt (stored 0%)
  adding: im0003.txt (stored 0%)
  adding: im0004.txt (deflated 2%)
  adding: im0005.txt (deflated 25%)
  adding: im0006.txt (deflated 20%)
  adding: im0007.txt (stored 0%)
  adding: im0008.txt (stored 0%)
  adding: im0010.txt (deflated 21%)
  adding: im0011.txt (stored 0%)
  adding: im0012.txt (stored 0%)
  adding: im0013.txt (deflated 2%)
  adding: im0014.txt (deflated 33%)
  adding: im0015.txt (stored 0%)
  adding: im0016.txt (deflated 27%)
  adding: im0018.txt (stored 0%)
  adding: im0019.txt (stored 0%)
  adding: im0020.txt (stored 0%)
  adding: im0023.txt (deflated 25%)
  adding: im0024.txt (deflated 39%)
  adding: im0025.txt (stored 0%)
  adding: im0027.txt (stored 0%)
  adding: im0028.txt (stored 0%)
  adding: im0029.txt (stored 0%)
  adding: im0033.txt (stored 0%)
  adding: im0034.txt (deflated 48%)
  adding: im0035.txt (deflated 27%)
  adding: im0036.txt (deflated 20%)
  adding: im0037.txt (stored 0%)
  addi

In [ ]:
%ls /content/drive/MyDrive/PaddleOCR/submit | wc -l

449
